In [77]:
import os
import json
from pydantic import BaseModel

from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

_ = load_dotenv(override=True)

In [78]:
from typing import Type, Any, get_origin, get_args
from pydantic import Field, BaseModel
from typing import List, Dict
import re

class ProductModel(BaseModel):
    """
    Product data model representing a product with its basic attributes.
    Attributes:
        name: The name of the product
        price: The price of the product without currency symbol
        features: A text list of product features or characteristics
    """
    name: str
    price: float
    features: List[str]

In [79]:
class CasualChatResponse(BaseModel):
    content: str = Field(description="The assistant text response")
    emotion: str = Field(
        description='The assistant emotion that should match the content of the response; must be one of ["neutral", "happy", "sad", "angry", "surprised", "fearful", "disgusted"]'
    )

In [80]:
CasualChatResponse.model_json_schema()

{'properties': {'content': {'description': 'The assistant text response',
   'title': 'Content',
   'type': 'string'},
  'emotion': {'description': 'The assistant emotion that should match the content of the response; must be one of ["neutral", "happy", "sad", "angry", "surprised", "fearful", "disgusted"]',
   'title': 'Emotion',
   'type': 'string'}},
 'required': ['content', 'emotion'],
 'title': 'CasualChatResponse',
 'type': 'object'}

In [83]:
schema = CasualChatResponse.model_json_schema()
template = {}

for field_name, field_schema in schema.get("properties", {}).items():
    description = field_schema.get("description", f"{field_name} value")
    if field_schema.get("type") == "array":
        template[field_name] = [description]
    else:
        template[field_name] = description
        

print(json.dumps(template, indent=4))

{
    "content": "The assistant text response",
    "emotion": "The assistant emotion that should match the content of the response; must be one of [\"neutral\", \"happy\", \"sad\", \"angry\", \"surprised\", \"fearful\", \"disgusted\"]"
}


In [74]:
def parse_docstring_attributes(docstring: str) -> Dict[str, str]:
    """
    Parse a docstring to extract attribute descriptions.
    
    Args:
        docstring: The class docstring
        
    Returns:
        Dictionary mapping field names to their descriptions
    """
    if not docstring:
        return {}
    
    # Find the Attributes section
    match = re.search(r'Attributes:(.*?)(?:\n\n|$)', docstring, re.DOTALL)
    if not match:
        return {}
    
    attributes_section = match.group(1)
    
    # Extract field descriptions using regex
    field_pattern = re.compile(r'\s*(\w+):\s*(.+?)(?=\n\s*\w+:|$)', re.DOTALL)
    fields = {}
    
    for match in field_pattern.finditer(attributes_section):
        field_name = match.group(1)
        description = match.group(2).strip()
        fields[field_name] = description
        
    return fields


def generate_model_template(model: Type[BaseModel]) -> str:
    """
    Generate a template string using the model's docstring descriptions.
    
    Args:
        model: A Pydantic model class with documented attributes
        
    Returns:
        A formatted string template with descriptions as placeholders
    """
    # Parse the docstring
    field_descriptions = parse_docstring_attributes(model.__doc__)
    
    # Create template dictionary
    template = {}
    
    for field_name, field_info in model.model_fields.items():
        if field_name in field_descriptions:
            # For list types, wrap description in a list
            if get_origin(field_info.annotation) is list:
                template[field_name] = [field_descriptions[field_name]]
            else:
                template[field_name] = field_descriptions[field_name]
        else:
            # Fallback if no description is found
            template[field_name] = f"{field_name} value here"
    
    # Convert to formatted string with double braces
    import json
    return json.dumps(template, indent=4).replace('{', '{{').replace('}', '}}')

In [75]:
model_template = generate_model_template(CasualChatResponse)
print(model_template)

{{
    "content": "The assistant text response",
    "emotion": "The assistant emotion that should match the content of the response; must be one of [\"neutral\", \"happy\", \"sad\", \"angry\", \"surprised\", \"fearful\", \"disgusted\"]"
}}


In [76]:
# Initialize Groq LLM
llm = ChatGroq(
    api_key=os.environ.get('GROQ_API_KEY'),
    model_name="llama-3.3-70b-versatile",
    temperature=0.7
)

# Define the expected JSON structure
parser = PydanticOutputParser(pydantic_object=CasualChatResponse)

# Create a simple prompt
prompt = ChatPromptTemplate.from_messages([
    ("user", "{input}"),
    ("user", f"""Answer the previous message in JSON with this structure:
        {model_template}"""),
])

# Create the chain that guarantees JSON output
chain = prompt | llm | parser
       
# Example usage
user_input = """Oye, acabo de escuchar un ruido raro y estoy solo en la casa, tengo miedo. ¿Tu no tienes miedo?"""

result = chain.invoke({"input": user_input})
result

<class '__main__.CasualChatResponse'>


CasualChatResponse(content='No te preocupes, estoy aquí para ayudarte. Como soy una inteligencia artificial, no tengo la capacidad de sentir emociones como el miedo. ¿Quieres que hablemos sobre lo que podrías hacer para sentirte más seguro en tu casa?', emotion='neutral')